Capstone Project

This file is only for creating plots, tables, and others from data required for making project pdf files.

script for debug & testing python files

In [3]:
import utility as util
import process as proc
from IPython.display import display # Allows the use of display() for DataFrames
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

# Read data from CSV files
mkt_df = proc.read_mkt_data('mkt_price_volume.csv')
# Display a description of the dataset
#display(mkt_df.describe())

# Fill missing value
data_before = proc.fill_missing_values(mkt_df)

# Divide data into categories(stock index, commodity, interest rates)
stock_price = data_before[['SPY:Price', 'SPF:Price', 'NAS:Price', 'DAX:Price', 'NIK:Price']]
commodity_price = data_before[['GLF:Price', 'OLF:Price']]
rate_price = data_before[['FFR:Price']]

# Preprocess: Price to Daily Returns, this also changes volume to daily increase
data_after = proc.price_to_return(data_before, 'Price', 'Return')

# Divide data into categories(stock index, commodity, interest rates)
stock_return = data_after[['SPY:Return', 'SPF:Return', 'NAS:Return', 'DAX:Return', 'NIK:Return']]
commodity_return = data_after[['GLF:Return', 'OLF:Return']]
rate_return = data_after[['FFR:Return']]

train_range = ["2010-01-01", "2015-12-31"]
test_range = ["2016-01-01", "2016-12-31"]

data_train = data_after.ix[train_range[0]:train_range[1]]
data_test = data_after.ix[test_range[0]:test_range[1]]

# Normalize data
#Save data to csv file

data_train_normal = proc.normalize_data(data_train)
data_test_normal = proc.normalize_data(data_test)

features_train = data_train_normal[:-1]
features_train = features_train.reset_index(drop=True)
features_test = data_test_normal[:-1]
features_test = features_test.reset_index(drop=True)

# Extract target
target_train = data_train['SPY:Return'][1:]
target_train = target_train.reset_index(drop=True)
target_train = proc.target_cluster_two(target_train, 1.0) 
target_train = target_train.astype(int)

target_test = data_test['SPY:Return'][1:]
target_test = target_test.reset_index(drop=True)
target_test = proc.target_cluster_two(target_test, 1.0) 
target_test = target_test.astype(int)

import run

# Fit the training data to the model using grid search
clf = run.fit_svm(features_train, target_train)
# Produce the value for 'max_depth'
#print "Parameter 'n_neighbors' is {} for the optimal model.".format(clf.get_params()['n_neighbors'])
print run.get_accuracy_score(clf, features_test, target_test)

0.521912350598


In [6]:
import quandl
import utility as util
import process as proc

quandl.ApiConfig.api_key = "dmDantC6pfu5ZsSZ9Zny"

date_range = ["2013-01-01", "2013-12-31"]
#spy: S&P500 index
spy = quandl.get("YAHOO/INDEX_GSPC", start_date=date_range[0], end_date=date_range[1])
spy = spy.rename(columns = {'Volume':'SPY:Volume', 'Adjusted Close':'SPY:Price'})
#Set date index from spy as a reference
date_index = spy.index.get_values()

#Get processed dataframe
df = proc.get_mkt_data(date_index, spy)

# Fill missing value
data = proc.fill_missing_values(df)
#util.save_data('test.csv',df)
price = data['SPY:Price']
volume = data['SPY:Volume']

cumulative_return = proc.get_cumulative_return(price, 5)
label = cumulative_return.copy()
for idx, val in cumulative_return.iteritems():
    if val >= 1.0:
        label.loc[idx] = 1
    else:
        label.loc[idx] = 0

#util.plot_data(price)
#util.plot_data(label)

print df.head()
df_transpose = df.transpose()
print df_transpose

              SPY:Volume    SPY:Price
2013-01-02  4.202600e+09  1462.420044
2013-01-03  3.829730e+09  1459.369995
2013-01-04  3.424290e+09  1466.469971
2013-01-07  3.304970e+09  1461.890015
2013-01-08  3.601600e+09  1457.150024
              2013-01-02    2013-01-03    2013-01-04    2013-01-07  \
SPY:Volume  4.202600e+09  3.829730e+09  3.424290e+09  3.304970e+09   
SPY:Price   1.462420e+03  1.459370e+03  1.466470e+03  1.461890e+03   

              2013-01-08    2013-01-09    2013-01-10    2013-01-11  \
SPY:Volume  3.601600e+09  3.674390e+09  4.081840e+09  3.340650e+09   
SPY:Price   1.457150e+03  1.461020e+03  1.472120e+03  1.472050e+03   

              2013-01-14    2013-01-15      ...         2013-12-17  \
SPY:Volume  3.003010e+09  3.135350e+09      ...       3.270030e+09   
SPY:Price   1.470680e+03  1.472340e+03      ...       1.781000e+03   

              2013-12-18    2013-12-19    2013-12-20    2013-12-23  \
SPY:Volume  4.327770e+09  3.497210e+09  5.097700e+09  2.851540e+09   

In [46]:
import process as proc
import numpy as np

spy = proc.read_mkt_data('mkt_price_volume.csv')
spy = spy['SPY:Price']
spy.name = 'SPY'
# Preprocess data
stocks = proc.read_stock_data('S&P500_stocks_price.csv')
stocks = proc.fill_missing_values(stocks)

stocks = stocks[['AAP', 'AES', 'AET']]
stocks = stocks.join(spy)

List of companies dropped due to insufficient data
ABBV
ACE
ADT
GAS
ARG
AA
ALLE
GOOG
ALTR
BXLT
BRCM
CVC
CAM
CCE
CPGX
CMCSK
CSC
CNX
DLPH
EMC
ESV
FB
FOSL
GM
GNW
HCA
HCBK
GMCR
KMI
KHC
LM
LYB
MNK
MPC
MHFI
WRK
KORS
NAVI
NWSA
NWS
NLSN
OI
PYPL
POM
PSX
PCL
PCP
QRVO
SNDK
SIAL
HOT
TE
TGNA
THC
TWC
TRIP
TYC
WBA
ANTM
XYL
ZBH
ZTS


In [55]:
import pandas as pd
import process as proc

df = pd.DataFrame(columns=['beta','alpha', 'corr'])

daily_return = proc.get_daily_returns(stocks)
corr = daily_return.corr()

for col in daily_return:
    beta, alpha = np.polyfit(daily_return['SPY'], daily_return[col], 1)
    df.ix[col] = [beta, alpha, corr['SPY'][col]]
    
print df
print corr

         beta     alpha      corr
AAP  0.757861  0.242781  0.431116
AES  1.200849 -0.201252  0.662522
AET  0.991436  0.009065  0.579669
SPY  1.000000  0.000000  1.000000
          AAP       AES       AET       SPY
AAP  1.000000  0.278103  0.257407  0.431116
AES  0.278103  1.000000  0.355169  0.662522
AET  0.257407  0.355169  1.000000  0.579669
SPY  0.431116  0.662522  0.579669  1.000000


In [65]:
from datetime import timedelta

print data.index[0] + timedelta(days=10)

2013-01-12 00:00:00


In [1]:
import pandas as pd
import utility as util
import process as proc
from IPython.display import display # Allows the use of display() for DataFrames
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import run
from sklearn.ensemble import RandomForestRegressor

# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

day_1 = 1
day_5 = 5
day_10 = 10
day_20 = 20

# Read data from CSV files
raw_data = proc.read_mkt_data('mkt_price_volume.csv')

# Fill missing value
raw_data = proc.fill_missing_values(raw_data)
price_data = proc.extract_columns(raw_data, 'Price')
price_data.drop('GLF:Price', axis = 1, inplace = True)
price_data.drop('NAS:Price', axis = 1, inplace = True)

day_1_return = proc.price_to_return(price_data, 'Price', 'Return(1D)', day_1)
day_5_return = proc.price_to_return(price_data, 'Price', 'Return(5D)', day_5)
day_10_return = proc.price_to_return(price_data, 'Price', 'Return(10D)', day_10)
day_20_return = proc.price_to_return(price_data, 'Price', 'Return(20D)', day_20)

#target
target = day_10_return['SPY:Return(10D)'].shift(-5)
target.name = target.name.replace('Return(10D)', 'Target')

#features
df = day_10_return
df = df.join(day_1_return['SPY:Return(1D)'])
df = df.join(day_5_return['SPY:Return(5D)'])
df = df.join(day_20_return['SPY:Return(20D)'])

train_range = ["2010-01-01", "2015-12-31"]

X = df[day_20:-5]
y = target[day_20:-5]

X_train = X.ix[train_range[0]:train_range[1]]
y_train = y.ix[train_range[0]:train_range[1]]

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

reg = RandomForestRegressor(n_estimators=30, max_depth=3)
reg = reg.fit(X_train, y_train)

#Stock datas
spy = proc.read_mkt_data('mkt_price_volume.csv')
spy = spy[['SPY:Price']]
spy = spy.rename(columns = {'SPY:Price':'SPY'})
# Preprocess data
stocks = proc.read_stock_data('S&P500_stocks_price.csv')
stocks = proc.fill_missing_values(stocks)
stocks = spy.join(stocks)
stocks = stocks[day_20:-5]

List of companies dropped due to insufficient data
ABBV
ACE
ADT
GAS
ARG
AA
ALLE
GOOG
ALTR
BXLT
BRCM
CVC
CAM
CCE
CPGX
CMCSK
CSC
CNX
DLPH
EMC
ESV
FB
FOSL
GM
GNW
HCA
HCBK
GMCR
KMI
KHC
LM
LYB
MNK
MPC
MHFI
WRK
KORS
NAVI
NWSA
NWS
NLSN
OI
PYPL
POM
PSX
PCL
PCP
QRVO
SNDK
SIAL
HOT
TE
TGNA
THC
TWC
TRIP
TYC
WBA
ANTM
XYL
ZBH
ZTS


In [2]:
simulation_range = ["2016-01-01", "2016-12-31"]
run.simulation(reg, X, y, stocks, simulation_range)

2016-01-05T00:00:00.000000000
1.00999150825
2016-01-06T00:00:00.000000000
1.00616493261
2016-01-07T00:00:00.000000000
0.996002401072
2016-01-08T00:00:00.000000000
0.994977272729
2016-01-11T00:00:00.000000000
1.00234746664
2016-01-12T00:00:00.000000000
1.00748990905
2016-01-13T00:00:00.000000000
0.998405943264
2016-01-14T00:00:00.000000000
1.0045575625
2016-01-15T00:00:00.000000000
0.99356983485
2016-01-19T00:00:00.000000000
1.00396349852
2016-01-20T00:00:00.000000000
0.987161101872
2016-01-21T00:00:00.000000000
0.993043036977
2016-01-22T00:00:00.000000000
1.01010533826
2016-01-25T00:00:00.000000000
1.00491318287
2016-01-26T00:00:00.000000000
1.0260600599
2016-01-27T00:00:00.000000000
1.02395027926
2016-01-28T00:00:00.000000000
1.01337010127
2016-01-29T00:00:00.000000000
1.04555122197
2016-02-01T00:00:00.000000000
1.03727820812
2016-02-02T00:00:00.000000000
0.981621138172
2016-02-03T00:00:00.000000000
0.98012244272
2016-02-04T00:00:00.000000000
0.99054698633
2016-02-05T00:00:00.00000000

In [72]:
import process as proc
import run

ret = proc.get_daily_returns(stocks)
#print stocks
#symbols = get_low_beta_stocks(stocks, today, 252, 3, 0.6)
weights = run.optimize_weight(stocks, ['SPY', 'MMM', 'ABT'], '2012-01-04', 252)


print sum(weights)
print weights
print ret.ix['2012-01-04'].tolist()
print sum(ret.ix['2012-01-04'].tolist()*weights)

1.0
[ 0.066  0.     0.934]
[1.0001879238163536, 1.0082644628101431, 0.9961212976011682]
0.996389694931


In [9]:
import run
import process as proc

stock_return = proc.get_daily_returns(stocks)
symbols = run.get_high_beta_stocks(stocks, '2016-01-04', 252, 5, 0.6)
ret = stock_return[symbols]
print ret.ix

                ETFC       LNC        MS       PFG      SCHW
2010-02-02  1.000000  1.000000  1.000000  1.000000  1.000000
2010-02-03  1.006536  0.973252  0.993942  0.977083  0.996709
2010-02-04  0.941558  0.933255  0.954823  0.962473  0.968079
2010-02-05  1.006897  0.997897  1.023658  0.995126  1.027288
2010-02-08  1.020548  0.990725  0.975789  0.987979  0.980631
